In [ ]:
from json.decoder import JSONDecodeError
import subprocess
import json
import base64
import socket
import time

#--------------Connect---------------
HOST = ""  
PORT = 5555

print("Waiting for connection...")
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((HOST, PORT))
s.listen()
conn, addr = s.accept()
conn.setblocking(0)

print('Connected by', addr)
connected = True
#-------------------------------------

#-------------Run Base Function---------------
TargetTracker = subprocess.Popen(['/home/m5stack/payload/bin/target_tracker'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
TargetTracker.stdin.write("_{\"stream\":1}\r\n".encode('utf-8'))
TargetTracker.stdin.flush()
#---------------------------------------------


img = ""
x = "0"
y = "0"
w = "0"
h = "0"

def TargetTracker_getROI(dataIn):
    ROI = dataIn.split(",")
    try:
        if ROI[0] == 'ROI':
            roiX = ROI[1]
            roiY = ROI[2]
            roiW = ROI[3]
            roiH = ROI[4]
            print("Received ROI: ", roiX, " ", roiY, " ", roiW, " ", roiH)
            TargetTracker_SetROI(int(roiX), int(roiY), int(roiW), int(roiH))            
    except Exception as e: 
        print(e)
        pass
    
def TargetTracker_SetROI(x, y, w, h):
    roi = { 
         'config':"Target Tracker",
              'x': x,
              'y': y,
              'w': w,
              'h': h
        }
    roi = json.dumps(roi)
    roi = roi.encode('utf-8')
    
    TargetTracker.stdin.write(roi)
    TargetTracker.stdin.flush()
    TargetTracker.stdin.write("_{\"stream\":1}\r\n".encode('utf-8'))
    TargetTracker.stdin.flush()
    
def TargetTracker_CheckXYWH(doc):
    global x, y, w, h
    if ('x' in doc) and ('y' in doc) and ('w' in doc) and ('h' in doc):
        x = str(doc['x'])
        y = str(doc['y'])
        w = str(doc['w'])
        h = str(doc['h'])
    else:
        x = "0"
        y = "0"
        w = "0"
        h = "0"
#     print("Target: ", x, " ", y, " ", w, " ", h)

def TargetTracker_CheckImg(doc):
    global img
    if ('img' in doc):
        img = doc['img']
        
def TargetTracker_SendUpdate():
    global conn
    conn.sendall(x.encode())
    conn.sendall(','.encode())
    conn.sendall(y.encode())
    conn.sendall(','.encode())
    conn.sendall(w.encode())
    conn.sendall(','.encode())
    conn.sendall(h.encode())
    conn.sendall(','.encode())
    conn.sendall(img.encode())
    conn.sendall('\n'.encode())
    
while True:
    try:
        doc = json.loads(TargetTracker.stdout.readline().decode('utf-8')) 

        TargetTracker_CheckXYWH(doc)
        TargetTracker_CheckImg(doc)   

        if connected:
            try:
                TargetTracker_SendUpdate()
            except Exception as e:
                if e.errno == 32 or e.errno == 104:
                    print("Connection lost")
                    conn.detach();
                    connected = False
        else:
            print("Waiting for connection...")
            conn, addr = s.accept()
            conn.setblocking(0)
            print('Connected by', addr) 
            connected = True
            
    except Exception as e: 
        print(e)
        pass
    
    try:
        dataIn = conn.recv(1024)
        dataIn = dataIn.decode('utf-8')
        TargetTracker_getROI(dataIn)
    except Exception as e: 
        pass